In [10]:
import os,django
import gensim
import pandas as pd

In [2]:
print (os.getcwd())

/Users/litianqi/school/TLCAS/papers/static/doc2vec_recommendation


### 获取上层目录，用于读取数据集

In [5]:
print (os.path.abspath(os.path.join(os.getcwd(), "../../..")))

/Users/litianqi/school/TLCAS


In [7]:
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

In [8]:
print (os.getcwd())

/Users/litianqi/school/TLCAS


### 读取数据库

In [11]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "TLCAS.settings")
django.setup()

In [13]:
from papers.models import *

In [15]:
try_list = list(PaperInfo.objects.all())

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

In [16]:
from asgiref.sync import sync_to_async

papers = sync_to_async(PaperInfo.objects.all)()

for paper in papers:
    print(paper)

TypeError: 'coroutine' object is not iterable